In [79]:
import pandas as pd
import numpy as np

df = pd.read_csv('fetched_data_weather.csv')

In [80]:
X = df.iloc[:,:6].values
Y = df.iloc[:,6:len(df.columns)].values

Stratified cross-validation 

In [81]:
from sklearn.model_selection import StratifiedKFold
k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

In [82]:
for fold, (train_idx, test_idx) in enumerate(skf.split(X, np.argmax(Y, axis=1))):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

In [83]:
y_train.shape

(38923, 9)

First, I want to try this:

In [84]:
from lazypredict.Supervised import LazyClassifier

In [85]:
# clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
# models,predictions = clf.fit(X_train, X_test, y_train, y_test)

# print(models)

God damn model is not working, anyway...
Here we go try KNN first

### KNN

In [86]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [111]:
df

,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,Condition_Drizzle,Condition_Fair,Condition_Foggy,Condition_Mixed,Condition_Mostly Cloudy,Condition_Partly Cloudy,Condition_Rain Shower,Condition_Thunderstorms,Condition_Wintry
0,37,34,87,3,0,30.08,0,0,0,0,0,1,0,0,0
1,45,37,76,9,0,30.08,0,0,0,0,1,0,0,0,0
2,45,39,81,8,0,30.08,0,0,0,0,1,0,0,0,0
3,43,36,76,6,0,30.05,0,0,0,0,1,0,0,0,0
4,41,36,81,5,0,30.05,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48648,48,39,71,13,0,30.18,0,0,0,0,1,0,0,0,0
48649,48,39,71,12,0,30.18,0,1,0,0,0,0,0,0,0
48650,48,39,71,10,0,30.18,0,1,0,0,0,0,0,0,0
48651,48,39,71,12,0,30.18,0,1,0,0,0,0,0,0,0


Accuracy of model?

In [87]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.561459403905447


If it was a regression problem it would probably be worked

In [88]:
# from sklearn.metrics import r2_score
# r2_score(y_test, y_pred)

So, the accuracy of model is 0.6

Are we satisfied? Hıh...
Let's check if KNN does better than this...

### KNN Model Tuning

In [89]:
knn_params = {
    'n_neighbors': list(range(3, 21, 2))
}

In [90]:
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()
knn_cv_model = GridSearchCV(knn, knn_params, cv=5, verbose=1, n_jobs=-1)

In [91]:
knn_cv_model.fit(X=X_train, y=y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


KeyboardInterrupt: 

In [ ]:
knn_cv_model.best_params_

{'n_neighbors': 7}

In [ ]:
knn_tuned = KNeighborsClassifier(n_neighbors=7)
knn_tuned.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=7)

In [ ]:
y_pred = knn_tuned.predict(X_test)
accuracy_score(y_test, y_pred)

0.561459403905447

### LightGBM

In [93]:
from lightgbm import LGBMClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

lgbm = LGBMClassifier()
scaler = StandardScaler()
skf = StratifiedKFold(
    n_splits = 5,
    shuffle=True,
    random_state = 23
)


Reminder: We can not use dummy variables which cause 2D array in LGBM

In [94]:
y_no_dummies = np.argmax(Y, axis=1)

In [95]:
total_acc = 0
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y_no_dummies)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_no_dummies[train_idx], y_no_dummies[test_idx]

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    lgbm.fit(X_train_scaled, y_train)
    y_pred = lgbm.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    total_acc += acc

    print(f"Fold {fold+1}: Accuracy = {acc}")

avg_acc = total_acc / k
print(f"\nAverage Accuracy = {avg_acc}")

Fold 1: Accuracy = 0.6837940602199157
Fold 2: Accuracy = 0.6745452677011612
Fold 3: Accuracy = 0.6721816873908129
Fold 4: Accuracy = 0.6729701952723536
Fold 5: Accuracy = 0.6813977389516958

Average Accuracy = 0.6769777899071878


#### LGBM Model Tuning 

In [105]:
lgbm_params = {
    'learning_rate':[0.03,0.05,0.07],
    'max_depth':[3,4,5],
    'min_child_samples':[3,5,10],
    'feature_fraction':[0.7, 0.8],
    'n_estimators': [30,50,70],
}

In [106]:
lgbm_cv_model = GridSearchCV( lgbm, lgbm_params, cv=10, verbose=2, n_jobs=4)

In [107]:
lgbm_cv_model.fit(X_train, y_train)

Fitting 10 folds for each of 162 candidates, totalling 1620 fits
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8


GridSearchCV(cv=10, estimator=LGBMClassifier(), n_jobs=4,
             param_grid={'feature_fraction': [0.7, 0.8],
                         'learning_rate': [0.03, 0.05, 0.07],
                         'max_depth': [3, 4, 5],
                         'min_child_samples': [3, 5, 10],
                         'n_estimators': [30, 50, 70]},
             verbose=2)

In [108]:
lgbm_best_params = lgbm_cv_model.best_params_
lgbm_best_model = lgbm_cv_model.best_estimator_

In [109]:
lgbm_best_model

LGBMClassifier(feature_fraction=0.8, learning_rate=0.05, max_depth=5,
               min_child_samples=10, n_estimators=30)

In [110]:
total_acc = 0
for fold, (train_idx, test_idx) in enumerate(skf.split(X, y_no_dummies)):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y_no_dummies[train_idx], y_no_dummies[test_idx]

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    lgbm_best_model.fit(X_train_scaled, y_train)
    y_pred = lgbm_best_model.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    total_acc += acc

    print(f"Fold {fold+1}: Accuracy = {acc}")

avg_acc = total_acc / k
print(f"\nAverage Accuracy = {avg_acc}")

Fold 1: Accuracy = 0.6455657178090638
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
Fold 2: Accuracy = 0.6376528619874627
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
Fold 3: Accuracy = 0.6414551433562841
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
Fold 4: Accuracy = 0.6411099691675232
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
Fold 5: Accuracy = 0.6453237410071943

Average Accuracy = 0.6422214866655056


Okay we could not catched better solution with tuning but at least we know why:
- We could use wider range model params
- We could give higher cv number into gridsearchcv argumant
- We could give n_jobs=-1 into gridsearchcv argumant 

But these cause longer model traning time

### CatBoost

In [25]:
# 1. import model that'll be used
from catboost import CatBoostClassifier
# 2. assign the model to a abbreviated variable
cat = CatBoostClassifier()
# 3. use fit method of model which takes X and Y variables
cat.fit(X_train, y_train_no_dummies)
# 4. use predict method of model and assign that to variable y_pred
y_pred = cat.predict(X_test)
# 5. check y_pred's accuracy
accuracy_score(y_test_no_dummies, y_pred) #0.73

Learning rate set to 0.095373
0:	learn: 1.9188816	total: 167ms	remaining: 2m 46s
1:	learn: 1.7500953	total: 189ms	remaining: 1m 34s
2:	learn: 1.6280781	total: 217ms	remaining: 1m 12s
3:	learn: 1.5317673	total: 242ms	remaining: 1m
4:	learn: 1.4581022	total: 264ms	remaining: 52.6s
5:	learn: 1.3941871	total: 288ms	remaining: 47.7s
6:	learn: 1.3419001	total: 310ms	remaining: 44s
7:	learn: 1.2964920	total: 333ms	remaining: 41.3s
8:	learn: 1.2574980	total: 357ms	remaining: 39.3s
9:	learn: 1.2241328	total: 382ms	remaining: 37.8s
10:	learn: 1.1941343	total: 404ms	remaining: 36.4s
11:	learn: 1.1678356	total: 427ms	remaining: 35.2s
12:	learn: 1.1454196	total: 453ms	remaining: 34.4s
13:	learn: 1.1236481	total: 474ms	remaining: 33.4s
14:	learn: 1.1058463	total: 497ms	remaining: 32.7s
15:	learn: 1.0888690	total: 517ms	remaining: 31.8s
16:	learn: 1.0741776	total: 536ms	remaining: 31s
17:	learn: 1.0606887	total: 558ms	remaining: 30.4s
18:	learn: 1.0487477	total: 581ms	remaining: 30s
19:	learn: 1.0385

0.6789311408016444

CatBoost Model Tuning

In [31]:
catb_params = {
    'learning_rate': [0.06,0.05,0.04], #updated
    'iterations': [70,80,90], #updated
    'depth': [7,8,10] #updated
}

In [32]:
cat_cv_model = GridSearchCV(cat, catb_params, cv = 10, verbose = 2, n_jobs=-1)
# for argument cv=3, model has overfitted 
cat_cv_model.fit(X_train, y_train_no_dummies)
cat_cv_model.best_params_

Fitting 10 folds for each of 27 candidates, totalling 270 fits


KeyboardInterrupt: 

In [28]:
cat = CatBoostClassifier(
    depth=7,
    iterations=80,
    learning_rate=0.05
)
cat_tuned = cat.fit(X_train,y_train_no_dummies)
y_pred = cat_tuned.predict(X_test)
accuracy_score(y_test_no_dummies, y_pred)

0:	learn: 2.0458783	total: 47.8ms	remaining: 3.77s
1:	learn: 1.9344562	total: 91.1ms	remaining: 3.55s
2:	learn: 1.8403680	total: 125ms	remaining: 3.21s
3:	learn: 1.7610464	total: 158ms	remaining: 2.99s
4:	learn: 1.6928448	total: 189ms	remaining: 2.84s
5:	learn: 1.6318526	total: 222ms	remaining: 2.73s
6:	learn: 1.5792813	total: 258ms	remaining: 2.69s
7:	learn: 1.5332058	total: 291ms	remaining: 2.62s
8:	learn: 1.4924545	total: 326ms	remaining: 2.57s
9:	learn: 1.4542621	total: 361ms	remaining: 2.53s
10:	learn: 1.4212965	total: 395ms	remaining: 2.48s
11:	learn: 1.3898625	total: 428ms	remaining: 2.42s
12:	learn: 1.3613324	total: 465ms	remaining: 2.4s
13:	learn: 1.3357869	total: 504ms	remaining: 2.38s
14:	learn: 1.3110680	total: 547ms	remaining: 2.37s
15:	learn: 1.2870436	total: 585ms	remaining: 2.34s
16:	learn: 1.2666365	total: 630ms	remaining: 2.33s
17:	learn: 1.2474900	total: 675ms	remaining: 2.32s
18:	learn: 1.2298040	total: 714ms	remaining: 2.29s
19:	learn: 1.2130764	total: 751ms	remain

0.6397738951695786

accuracy: 0.6397: overfitted 

it takes so much time if you want to get better accuracy, my GPU is crying.

#### ChatGBT considered about this situation that tuned data give me worse accuracy:

It is possible to encounter situations where hyperparameter tuning results in a lower accuracy score than the default hyperparameters. This can happen because of overfitting on the training data, especially if the tuning process was not performed properly or if the dataset is small. In this case, the best hyperparameters found during tuning may not generalize well to unseen data.

It's also possible that the original default hyperparameters of the CatBoost model were already well-suited for your dataset, and that the hyperparameter tuning process actually made the model worse by overfitting.

To further investigate this issue, you can try the following steps:

Re-run the model with the original default hyperparameters and see if the accuracy score matches the initial score of 0.73.

Try reducing the complexity of the model by decreasing the value of the 'depth' parameter, and see if that improves the accuracy score.

Try increasing the size of the dataset, or implementing other techniques such as cross-validation or early stopping to prevent overfitting during hyperparameter tuning.

Check if there were any changes made to the data preprocessing or feature engineering steps between the initial model training and the tuning process. Any changes made to the data may affect the performance of the model.

By performing these steps, you may be able to identify the reason for the decrease in accuracy score after hyperparameter tuning and improve the overall performance of the model.

### ANN: Keras

In [33]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test) 

In [34]:
from keras.models import Sequential
from keras.layers import Dense

In [44]:
classifier = Sequential()

classifier.add(Dense(
    units = X_train_scaled.shape[1],
    activation = 'relu',
))

classifier.add(Dense(
    units = 6,
    activation = 'relu'
))

classifier.add(Dense(
    units = 8,
    activation = 'relu'
))

classifier.add(Dense(
    units = 9,
    activation = 'softmax',
))

In [45]:
classifier.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

In [46]:
y_train.shape

(38923, 9)

In [48]:
classifier.fit(X_train_scaled,y_train, epochs = 100, batch_size=16, verbose=1)

Epoch 1/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0126 - accuracy: 0.6158
Epoch 2/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0098 - accuracy: 0.6158
Epoch 3/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0103 - accuracy: 0.6146
Epoch 4/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0093 - accuracy: 0.6148
Epoch 5/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0095 - accuracy: 0.6158
Epoch 6/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0092 - accuracy: 0.6164
Epoch 7/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0094 - accuracy: 0.6155
Epoch 8/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0087 - accuracy: 0.6157
Epoch 9/100
2433/2433 [==============================] - 5s 2ms/step - loss: 1.0094 - accuracy: 0.6158
Epoch 10/100
2433/2433 [==============================] - 6s 2ms/step - l

KeyboardInterrupt: 

### SVC

In [68]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

svc = SVC()
scaler = StandardScaler()

k = 5
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

total_acc = 0
for fold, (train_idx, test_idx) in enumerate(skf.split(X, np.argmax(Y, axis=1))):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    svc.fit(X_train_scaled, np.argmax(y_train, axis=1))
    y_pred = svc.predict(X_test_scaled)
    
    acc = accuracy_score(np.argmax(y_test, axis=1), y_pred)
    total_acc += acc
    
    print(f"Fold {fold+1}: Accuracy = {acc}")

avg_acc = total_acc / k
print(f"\nAverage Accuracy = {avg_acc}")

Fold 1: Accuracy = 0.6206967423697461
Fold 2: Accuracy = 0.6247045524612065
Fold 3: Accuracy = 0.6290206556366252
Fold 4: Accuracy = 0.623535457348407
Fold 5: Accuracy = 0.6250770811921891

Average Accuracy = 0.6246068978016348


- oneclasssvm
- isolationforest

- k-means

In [112]:
df

,Temperature,Dew Point,Humidity,Wind Speed,Wind Gust,Pressure,Condition_Drizzle,Condition_Fair,Condition_Foggy,Condition_Mixed,Condition_Mostly Cloudy,Condition_Partly Cloudy,Condition_Rain Shower,Condition_Thunderstorms,Condition_Wintry
0,37,34,87,3,0,30.08,0,0,0,0,0,1,0,0,0
1,45,37,76,9,0,30.08,0,0,0,0,1,0,0,0,0
2,45,39,81,8,0,30.08,0,0,0,0,1,0,0,0,0
3,43,36,76,6,0,30.05,0,0,0,0,1,0,0,0,0
4,41,36,81,5,0,30.05,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48648,48,39,71,13,0,30.18,0,0,0,0,1,0,0,0,0
48649,48,39,71,12,0,30.18,0,1,0,0,0,0,0,0,0
48650,48,39,71,10,0,30.18,0,1,0,0,0,0,0,0,0
48651,48,39,71,12,0,30.18,0,1,0,0,0,0,0,0,0
